<a href="https://colab.research.google.com/github/kittimaxz/Project_BoneAge/blob/main/Test_BoNet_RSNA_RHPE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Standard lib imports
import os
import csv
import glob
import time
import argparse
import warnings
import pandas as pd
import os.path as osp

In [2]:
# PyTorch imports
import torch
import torch.nn as nn
import torch.optim as optim
!pip install horovod
import horovod.torch as hvd
from torchvision import transforms
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torch.utils.data.distributed import DistributedSampler

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
# Other imports
from tqdm import tqdm
import pdb

In [4]:
warnings.filterwarnings("ignore")

In [5]:
parser = argparse.ArgumentParser()

In [6]:
# Dataloading-related settings
parser.add_argument('--heatmaps', default=False, action='store_true',
                help='Train model with gaussian heatmaps')
parser.add_argument('--cropped', default=False, action='store_true',
                help='Train model with cropped images according to bbox')
parser.add_argument('--dataset', default='RSNA', type=str,choices=['RSNA','RHPE'],
                help='Dataset to perform training')

parser.add_argument('--data-train', default='data/train/', type=str,
                help='path to train data folder')
parser.add_argument('--ann-path-train', default='train.csv', type=str,
                help='path to BAA annotations file')
parser.add_argument('--rois-path-train', default='train.json',
                type=str, help='path to ROIs annotations in coco format')

parser.add_argument('--data-val', default='data/val/', type=str,
                help='path to val data folder')
parser.add_argument('--ann-path-val', default='val.csv', type=str,
                help='path to BAA annotations file')
parser.add_argument('--rois-path-val', default='val.json',
                type=str, help='path to ROIs annotations in coco format')

parser.add_argument('--save-folder', default='TRAIN/new_test/',
                help='location to save checkpoint models')
parser.add_argument('--snapshot', default='boneage_bonet_weights.pth',
                help='path to weight snapshot file')
parser.add_argument('--optim-snapshot', type=str,
                default='boneage_bonet_optim.pth',
                help='path to optimizer state snapshot')

parser.add_argument('--eval-first', default=False, action='store_true',
                help='evaluate model weights before training')
parser.add_argument('-j', '--workers', default=4, type=int, metavar='N',
                help='number of data loading workers (default: 4)')

_StoreAction(option_strings=['-j', '--workers'], dest='workers', nargs=None, const=None, default=4, type=<class 'int'>, choices=None, help='number of data loading workers (default: 4)', metavar='N')

In [7]:
# Training procedure settings
parser.add_argument('--batch-size', default=1, type=int,
                help='Batch size for training')
parser.add_argument('--epochs', type=int, default=20,
                help='upper epoch limit')
parser.add_argument('--lr', '--learning-rate', default=1e-5, type=float,
                help='initial learning rate')
parser.add_argument('--patience', default=2, type=int,
                help='patience epochs for LR decreasing')
parser.add_argument('--start-epoch', type=int, default=1,
                help='epoch number to resume')
parser.add_argument('--seed', type=int, default=1111,
                    help='random seed')
parser.add_argument('--log-interval', type=int, default=30, metavar='N',
                    help='report interval')

parser.add_argument('--gpu', type=str, default='2,3')

_StoreAction(option_strings=['--gpu'], dest='gpu', nargs=None, const=None, default='2,3', type=<class 'str'>, choices=None, help=None, metavar=None)

In [13]:
args = parser.parse_args()

usage: ipykernel_launcher.py [-h] [--heatmaps] [--cropped]
                             [--dataset {RSNA,RHPE}] [--data-train DATA_TRAIN]
                             [--ann-path-train ANN_PATH_TRAIN]
                             [--rois-path-train ROIS_PATH_TRAIN]
                             [--data-val DATA_VAL]
                             [--ann-path-val ANN_PATH_VAL]
                             [--rois-path-val ROIS_PATH_VAL]
                             [--save-folder SAVE_FOLDER] [--snapshot SNAPSHOT]
                             [--optim-snapshot OPTIM_SNAPSHOT] [--eval-first]
                             [-j N] [--batch-size BATCH_SIZE]
                             [--epochs EPOCHS] [--lr LR] [--patience PATIENCE]
                             [--start-epoch START_EPOCH] [--seed SEED]
                             [--log-interval N] [--gpu GPU]
ipykernel_launcher.py: error: unrecognized arguments: -f /root/.local/share/jupyter/runtime/kernel-679485e7-58fc-415c-aab2-ddeff497f11

SystemExit: ignored

In [9]:
args_dict = vars(args)
print('Argument list to program')
print('\n'.join(['--{0} {1}'.format(arg, args_dict[arg])
                 for arg in args_dict]))
print('\n\n')

NameError: ignored